In [1]:
!pip install tqdm
!pip install tensorflow_hub
!pip install tensorflow==2.2.0
import os
import s3fs
import math
import tqdm
import boto3
import random
#import mxnet as mx
import urllib.request
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Model
from tensorflow.keras.applications import (MobileNetV2,ResNet50)
from tensorflow.keras.layers import (Dense,Dropout,Flatten,Input)
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

     |████████████████████████████████| 108 kB 17.3 MB/s            
     |████████████████████████████████| 516.2 MB 10 kB/s               
     |████████████████████████████████| 2.9 MB 45.5 MB/s            
     |████████████████████████████████| 4.3 MB 90.5 MB/s            
     |████████████████████████████████| 126 kB 81.8 MB/s            
     |████████████████████████████████| 454 kB 78.4 MB/s            
     |████████████████████████████████| 26.1 MB 51.1 MB/s            
     |████████████████████████████████| 42 kB 1.5 MB/s             
     |████████████████████████████████| 65 kB 6.1 MB/s              
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 3.0 MB 53.7 MB/s            
     |████████████████████████████████| 152 kB 104.8 MB/s            
     |████████████████████████████████| 781 kB 106.7 MB/s            
     |████████████████████████████████| 97 kB 11.4 MB/s            
     |████████████████████████████████| 155 kB 107.1 MB/s 

In [10]:
#parameters
batch_size = 128
input_size  = 112
embd_shape = 512
sub_name = 'arc_mobv2' #arc_rec50 or arc_mobv2
backbone_type = 'MobileNetV2' # 'ResNet50', 'MobileNetV2'
head_type = "ArcHead" # 'ArcHead', 'NormHead'
is_ccrop = False # central-cropping or not
binary_img = True
num_classes =  94873 # 9131 # 85743 #
num_samples = 8960460 # 3137806 # 5822653 # 
epochs = 5
base_lr = 0.01
w_decay = float(5e-4)
save_steps = 500
tfrecord_path = "data/ms1m_.tfrecord"


In [11]:
tf.__version__

'2.2.0'

In [12]:
#reading TFrecords
def _parse_tfrecord(binary_img=False, is_ccrop=False):
    def parse_tfrecord(tfrecord):
        if binary_img:
            features = {'image/source_id': tf.io.FixedLenFeature([], tf.int64),
#                         'image/data_source': tf.io.FixedLenFeature([], tf.string),
                        'image/encoded': tf.io.FixedLenFeature([], tf.string)}
            x = tf.io.parse_single_example(tfrecord, features)
            x_train = tf.image.decode_jpeg(x['image/encoded'], channels=3)
        else:
            features = {'image/source_id': tf.io.FixedLenFeature([], tf.int64),
                        'image/img_path': tf.io.FixedLenFeature([], tf.string)}
            x = tf.io.parse_single_example(tfrecord, features)
            image_encoded = tf.io.read_file(x['image/img_path'])
            x_train = tf.image.decode_jpeg(image_encoded, channels=3)

        y_train = tf.cast(x['image/source_id'], tf.float32)
        print(y_train)
        x_train = _transform_images(is_ccrop=is_ccrop)(x_train)
        y_train = _transform_targets(y_train)
        return (x_train, y_train), y_train
    return parse_tfrecord

def _transform_images(is_ccrop=False):
    def transform_images(x_train):
        x_train = tf.image.resize(x_train, (128, 128))
        x_train = tf.image.random_crop(x_train, (input_size, input_size, 3))
        x_train = tf.image.random_flip_left_right(x_train)
        x_train = tf.image.random_saturation(x_train, 0.6, 1.4)
        x_train = tf.image.random_brightness(x_train, 0.4)
        x_train = x_train / 255
        return x_train
    return transform_images

def _transform_targets(y_train):
    return y_train

def load_tfrecord_dataset(tfrecord_name, batch_size, binary_img=False, 
                          shuffle=True, buffer_size=10240,is_ccrop=False):
    """load dataset from tfrecord"""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()
    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)
    dataset = raw_dataset.map(
        _parse_tfrecord(binary_img=binary_img, is_ccrop=is_ccrop),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(
        buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset


In [13]:
def Backbone(backbone_type='ResNet50', use_pretrain
             =True):
    """Backbone Model"""
    weights = None
    if use_pretrain:
        weights = 'imagenet'

    def backbone(x_in):
        if backbone_type == 'ResNet50':
            return ResNet50(input_shape=x_in.shape[1:], include_top=False,
                            weights=weights)(x_in)
        elif backbone_type == 'MobileNetV2':
            return MobileNetV2(input_shape=x_in.shape[1:], include_top=False,
                               weights=weights)(x_in)
        else:
            raise TypeError('backbone_type error!')
    return backbone


def OutputLayer(embd_shape, w_decay=5e-4, name='OutputLayer'):
    """Output Later"""
    def output_layer(x_in):
        x = inputs = Input(x_in.shape[1:])
        x = BatchNormalization()(x)
        x = Dropout(rate=0.5)(x)
        x = Flatten()(x)
        x = Dense(embd_shape, kernel_regularizer=tf.keras.regularizers.l2(w_decay) )(x)
        x = BatchNormalization()(x)
        return Model(inputs, x, name=name)(x_in)
    return output_layer

def ArcHead(num_classes, margin=0.5, logist_scale=64, name='ArcHead'):
    """Arc Head"""
    def arc_head(x_in, y_in):
        x = inputs1 = Input(x_in.shape[1:])
        y = Input(y_in.shape[1:])
        x = ArcMarginPenaltyLogists(num_classes=num_classes,
                                    margin=margin,
                                    logist_scale=logist_scale)(x, y)
        return Model((inputs1, y), x, name=name)((x_in, y_in))
    return arc_head

def ArcFaceModel(size, channels, num_classes, name, 
                 margin, logist_scale, embd_shape, head_type, 
                 backbone_type, w_decay, use_pretrain, training):
    """Arc Face Model"""
    x = inputs = Input([size, size, channels], name='input_image')

    x = Backbone(backbone_type=backbone_type, use_pretrain=use_pretrain)(x)

    embds = OutputLayer(embd_shape, w_decay=w_decay)(x)
    if training:
        assert num_classes is not None
        labels = Input([], name='label')
        if head_type == 'ArcHead':
            logist = ArcHead(num_classes=num_classes, margin=margin,
                             logist_scale=logist_scale)(embds, labels)
        else:
            logist = NormHead(num_classes=num_classes, w_decay=w_decay)(embds)
        return Model((inputs, labels), logist, name=name)
    else:
        return Model(inputs, embds, name=name)
    
    #return Model(inputs, embds, name=name)

In [14]:
class BatchNormalization(tf.keras.layers.BatchNormalization):
    """Make trainable=False freeze BN for real (the og version is sad).
       ref: https://github.com/zzh8829/yolov3-tf2
    """
    def call(self, x, training=False):
        if training is None:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)
        return super().call(x, training)

class ArcMarginPenaltyLogists(tf.keras.layers.Layer):
    """ArcMarginPenaltyLogists"""
    def __init__(self, num_classes, margin=0.5, logist_scale=64, **kwargs):
        super(ArcMarginPenaltyLogists, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.margin = margin
        self.logist_scale = logist_scale
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            
            'num_classes': self.num_classes,
            'margin': self.margin,
            'logist_scale': self.logist_scale
        })
        return config
    
    def build(self, input_shape):
        self.w = self.add_variable(
            "weights", shape=[int(input_shape[-1]), self.num_classes])
        self.cos_m = tf.identity(math.cos(self.margin), name='cos_m')
        self.sin_m = tf.identity(math.sin(self.margin), name='sin_m')
        self.th = tf.identity(math.cos(math.pi - self.margin), name='th')
        self.mm = tf.multiply(self.sin_m, self.margin, name='mm')

    def call(self, embds, labels):
        normed_embds = tf.nn.l2_normalize(embds, axis=1, name='normed_embd')
        normed_w = tf.nn.l2_normalize(self.w, axis=0, name='normed_weights')

        cos_t = tf.matmul(normed_embds, normed_w, name='cos_t')
        sin_t = tf.sqrt(1. - cos_t ** 2, name='sin_t')

        cos_mt = tf.subtract(
            cos_t * self.cos_m, sin_t * self.sin_m, name='cos_mt')

        cos_mt = tf.where(cos_t > self.th, cos_mt, cos_t - self.mm)

        mask = tf.one_hot(tf.cast(labels, tf.int32), depth=self.num_classes,
                          name='one_hot_mask')

        logists = tf.where(mask == 1., cos_mt, cos_t)
        logists = tf.multiply(logists, self.logist_scale, 'arcface_logist')

        return logists

In [15]:
def SoftmaxLoss():
    """softmax loss"""
    def softmax_loss(y_true, y_pred):
        # y_true: sparse target
        # y_pred: logist
        y_true = tf.cast(tf.reshape(y_true, [-1]), tf.int32)
        ce = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
                                                            logits=y_pred)
        return tf.reduce_mean(ce)
    return softmax_loss


In [16]:
print("Loading model")
model = ArcFaceModel(size=input_size,
                     channels=3, 
                     num_classes=num_classes, 
                     name='arcface_model', 
                     margin=0.5, 
                     logist_scale=64, 
                     embd_shape=embd_shape, 
                     head_type=head_type, 
                     backbone_type = backbone_type,
                     w_decay=w_decay, 
                     use_pretrain = True, 
                     training=True)
model.summary(line_length=80)

Loading model
Model: "arcface_model"
________________________________________________________________________________
Layer (type)              Output Shape      Param #  Connected to               
input_image (InputLayer)  [(None, 112, 112, 0                                   
________________________________________________________________________________
mobilenetv2_1.00_224 (Mod (None, 4, 4, 1280 2257984  input_image[0][0]          
________________________________________________________________________________
OutputLayer (Model)       (None, 512)       10493440 mobilenetv2_1.00_224[1][0] 
________________________________________________________________________________
label (InputLayer)        [(None,)]         0                                   
________________________________________________________________________________
ArcHead (Model)           (None, 94873)     48574976 OutputLayer[1][0]          
                                                     label[0][0]        

In [17]:
def get_ckpt_inf(ckpt_path, steps_per_epoch):
    """get ckpt information"""
    split_list = ckpt_path.split('e_')[-1].split('_b_')
    epochs = int(split_list[0])
    batchs = int(split_list[-1].split('.ckpt')[0])
    steps = (epochs - 1) * steps_per_epoch + batchs

    return epochs, steps + 1

In [18]:
dataset_len = num_samples
#path = 9
steps_per_epoch = dataset_len // batch_size #==(261)
dataset = load_tfrecord_dataset(tfrecord_path, batch_size, binary_img= binary_img,is_ccrop=is_ccrop)
learning_rate = tf.constant(base_lr)
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
loss_fn = SoftmaxLoss()

Tensor("Cast:0", shape=(), dtype=float32)


In [19]:
print("[*] training from scratch.")
epochs, steps = 1, 1

[*] training from scratch.


In [23]:
model.compile(optimizer=optimizer, loss=loss_fn) #Try true
check_path = ('./ckpt/ms1m_vgg_ckpt/e_{epoch}_b_{loss}.ckpt')
mc_callback = ModelCheckpoint(check_path,save_freq=1000 , verbose=1, save_weights_only=True)
tb_callback = TensorBoard(log_dir='logs/', update_freq=batch_size * 5, profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
callbacks = [mc_callback, tb_callback]

history = model.fit(dataset, 
                    epochs=5, 
                    steps_per_epoch=steps_per_epoch, 
                    callbacks=callbacks, 
                    initial_epoch=epochs - 1)

print("[*] training done!")

Epoch 1/5
   12/70003 [..............................] - ETA: 44:24:54 - loss: 41.4361

KeyboardInterrupt: 

In [ ]:
model.save_weights('./stuf/keras.h5')

In [ ]:
os.makedirs('./model', exist_ok=True)
model.save('./model/keras_model.h5')

In [ ]:
Newmodel.compile(optimizer=optimizer, loss=loss_fn) #Try true

In [ ]:
Newmodel.load_weights('./stuf/keras.h5')

In [ ]:
model = tf.saved_model.load("Model/SavedModelFormat")
# model.count_params()

In [ ]:
# model.summary
from tensorflow.python.keras.utils.layer_utils import count_params

# trainable_count = count_params(model.trainable_weights)
# non_trainable_count = count_params(model.non_trainable_weights)

In [ ]:
m = tf.keras.Sequential([hub.KerasLayer("512_embed", trainable=True),
    tf.keras.layers.Dense(10, activation='softmax')
])
m.build([None, 112, 112, 3])
m.summary()

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()
open("converted_model.tflite", "wb").write(quantized_model)

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_no_quant_tflite = converter.convert()
MODEL_NO_QUANT_TFLITE = 'Model/96_embed/model_no_quant.tflite'
# # Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_no_quant_tflite = converter.convert()
MODEL_NO_QUANT_TFLITE = 'Model/checkpointss/model_no_quant.tflite'
# # Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

In [ ]:
import numpy as np
# Open TensorFlow ckpt
path = "ckpt/ms1m_vgg_ckpt/"
reader = tf.train.load_checkpoint(path)

print('\nCount the number of parameters in ckpt file(%s)' % path)
param_map = reader.get_variable_to_shape_map()
total_count = 0
for k, v in param_map.items():
    if 'Momentum' not in k and 'global_step' not in k:
        temp = np.prod(v)
        total_count += temp

print('Total Param Count: %d' % total_count)